In [122]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
from Crypto.Random import get_random_bytes
from sympy import randprime

In [123]:
#1. krok - odbiorca RSA
P = randprime(pow(2, 124), pow(2, 125))
Q = randprime(pow(2, 124), pow(2, 125))
N = P*Q
Fi = (P-1)*(Q-1)
E = 65537
D = pow(E, -1, Fi)
print("P:", str(P), "Q:", str(Q), "N:", str(N), "Fi:", str(Fi), "D:", str(D))

P: 30400720939726580239196672764046771443 Q: 23811013565199919472578333434626785591 N: 723871978687686845562844154989572065658716388559828487064889984452542677813 Fi: 723871978687686845562844154989572065604504654054901987353114978253869120780 D: 198582393225596560666102736813670387224062578013230432131798132261566945733


In [124]:
#2. krok - nadawca
# Dostaje TYLKO: E,N
#Szyfruje kluczem pub RSA od odbiorcy wiadomość

In [125]:
# 3. NADAWCA szyfrowanie wiadomośći przez  AES szyfrowanie wiadomości
W = b"XDDD"
KL = get_random_bytes(32)
nonce = get_random_bytes(12)
CIPHER = AES.new(KL, AES.MODE_EAX)
nonce = CIPHER.nonce
#electronic code book - najprostrzy tryb kodowania
KRYPTO = CIPHER.encrypt(pad(W, 16))

# W2 = unpad(CIPHER.decrypt(KRYPTO), 16)
# print(W2)

In [126]:
#4. SZYFROWANIE KLUCZA AES
KL_TAJNY = pow(int.from_bytes(KL, "big"),E,N)
#wiadomość szyfrowana AES (kryptogram) i klucz AES zaszyfrowany RSA jest wysyłane do odbiorcy
print("KL_TAJNY: ", str(KL_TAJNY))
print("KRYPTOGRAM WIADOMOŚĆI: ", str(KRYPTO))

KL_TAJNY:  671543140029822849879952352807845162621453916099339879810344673434128107475
KRYPTOGRAM WIADOMOŚĆI:  b'\xdbi\xccAe\x16\xe7\\\x18\\\xd7#"46`'


In [127]:
KL_ODSZ = pow(KL_TAJNY, D, N)
print(type(KL_ODSZ))
CIPHER = AES.new(KL_ODSZ.to_bytes(32, "big"), AES.MODE_EAX, nonce=nonce)

<class 'int'>


In [128]:
#5. ODBIORCA deszyfruje klucz AES swoim kluczem prywatnym. Potem deszyfruje wiadomość kluczem

W2 = unpad(CIPHER.decrypt(KRYPTO))
print(W2)

b'Q\x85"\x87\x81\xd4|4\xb0\xb3I\xa6D\xc7\xf0>'
